# cGAN (conditional GAN) implementation Using Pytorch

## Implementation Starts here

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as vutils

In [39]:
# Generator

class Generator(nn.Module):
    def __init__(self, input_size_z = 100, input_size_condition= 100, hidden_size = 256, output_size = 784, layers = 1, device = 'cuda'):
        super().__init__()

        self.device = device

        self.init_layer = nn.Sequential(
            nn.Linear(input_size_z +input_size_condition, 1200, device = self.device),
            nn.ReLU(),
        )

        self.combine = nn.Sequential(
            nn.Linear(1200, hidden_size, device = self.device),
            nn.ReLU(),
        )

        self.layer = nn.ModuleList([nn.Sequential(
            nn.Linear(hidden_size, hidden_size, device = self.device),
            nn.ReLU(),
        ) for _ in range(layers-1)])

        self.final = nn.Sequential(
            nn.Linear(hidden_size, output_size, device = self.device),
            nn.Sigmoid()
        )

    def forward(self, z, y):
        """
        z: the vector 
        y: the label for the vector
        """
        z = z.to(self.device)
        y = y.to(self.device)

        combined = self.init_layer(torch.cat((z, y), dim=1))
        combined = self.combine(combined)

        for layer in self.layer:
            combined = layer(combined)
            
        return self.final(combined)

In [40]:
def test_generator():
    gen = Generator(input_size_z=100, input_size_condition=10, output_size=784, layers=3, device='cuda')
    noise = torch.randn(1, 100)
    label = torch.randn(1, 10)

    output = gen(noise, label)
    assert output.shape == (1, 784)
    print("Generator test passed")
test_generator()

Generator test passed


In [50]:
# Discriminator - not using maxout because of instability issues

class Discriminator(nn.Module):
    def __init__(self, d_in =784, d_label = 10, hidden_size=256, d_out =1, leaky = 0.2, dropout = 0.5, device='cuda'):
        super().__init__()

        self.device = device

        self.dropout = nn.Dropout(dropout)

        self.label_embed = nn.Linear(d_label, hidden_size, device = self.device)

        self.model = nn.Sequential(
            nn.Linear(d_in + hidden_size, hidden_size, device = self.device),
            nn.LeakyReLU(leaky),
            nn.Linear(hidden_size, hidden_size, device = self.device),
            nn.LeakyReLU(leaky),
            nn.Linear(hidden_size, d_out, device = self.device)
        )

        
    def forward(self, x, y):
        x = x.to(self.device)
        y = y.to(self.device)

        y = self.label_embed(y)

        combined = torch.cat((x,y), dim =1)

        logits = self.dropout(self.model(combined))

        return logits # torch.sigmoid(combined) or torch.tanh(combined) - used with loss

In [51]:
# Unit testing for Discriminator

def test_discriminator():
   disc = Discriminator(d_in = 784, d_label = 10, hidden_size=256, d_out =1, leaky = 0.2, dropout = 0.5, device='cuda')
   x = torch.randn(1, 784)
   y = torch.randn(1, 10)

   output = disc(x, y)
   assert output.shape == (1, 1)
   print("Discriminator test passed")
test_discriminator()

Discriminator test passed


## Training

In [ ]:
#Hyperparameters
LR= 0.1
MIN_LR = 0.000001
DECAY_FACTOR = 1.00004
DROPOUT = 0.5
INIT_MOMENTUM = 0.5
MAX_MOMENTUM = 0.7
BATCH_SIZE = 100
EPOCHS = 10